In [3]:
import pandas as pd
import gzip
import os
from tqdm import tqdm
import time

# パス設定
normal_folder = "/root/user/j0114690/EGT/data_processed/normal"
anomaly_folder = "/root/user/j0114690/EGT/data_processed/anormal"
excel_file_path = "/root/user/j0114690/EGT/EGT_v6/anormal_info.xlsx"
#xlsx内に右記を含むこと：sheet_name="故障・仮故障", ラベル名=["VIN", "故障ig_cycle"]


def read_gz_files(folder_path, user_id_column):
    dataframes = {}
    
    start_time = time.time()
    
    # .pkl.gzファイルのみを対象とする
    pkl_gz_files = [file for file in os.listdir(folder_path) if file.endswith('.pkl.gz')]
    
    with tqdm(total=len(pkl_gz_files), desc=f"Reading files from {folder_path}", unit="file") as pbar:
        for file in pkl_gz_files:
            file_path = os.path.join(folder_path, file)
            
            with gzip.open(file_path, 'rb') as f:
                # .pkl.gzファイルを直接pandasで読み込む
                pandas_df = pd.read_pickle(f)
            
            pandas_df["total_ig_cycle"] = pandas_df["total_ig_cycle"].astype('int64')
            
            user_id = pandas_df[user_id_column].iloc[0]
            if user_id not in dataframes:
                dataframes[user_id] = []
            dataframes[user_id].append(pandas_df)
            
            pbar.update(1)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time for reading files from {folder_path}: {elapsed_time:.2f} seconds")
    
    return dataframes

def read_excel(file_path):
    excel_df = pd.read_excel(file_path, sheet_name="故障・仮故障", usecols=["VIN", "故障ig_cycle"])
    return excel_df

def extract_failure_data(failure_df, failure_range_list):
    extracted_data = {}
    vin_counts = {}
    
    for _, row in failure_range_list.iterrows():
        vin = row["VIN"]
        failure_ig_cycle = row["故障ig_cycle"]
        
        if vin in failure_df:
            user_df = pd.concat(failure_df[vin])
            user_df["total_ig_cycle"] = user_df["total_ig_cycle"].astype('int64')
            
            start_cycle = failure_ig_cycle - 20
            end_cycle = failure_ig_cycle
            
            mask = (user_df["total_ig_cycle"] >= start_cycle) & (user_df["total_ig_cycle"] <= end_cycle)
            extracted_user_df = user_df[mask]
            
            if not extracted_user_df.empty:
                if vin not in vin_counts:
                    vin_counts[vin] = 1
                    extracted_data[vin] = [extracted_user_df]
                else:
                    vin_counts[vin] += 1
                    new_vin = f"{vin}-{vin_counts[vin]}"
                    extracted_data[new_vin] = [extracted_user_df]
    
    return extracted_data

def print_extracted_ig_cycles(extracted_data):
    for vin, user_dfs in extracted_data.items():
        for user_df in user_dfs:
            ig_cycles = user_df["total_ig_cycle"].unique().tolist()
            print(f"VIN: {vin}, Extracted IG Cycles: {ig_cycles}")


# VIN(ユーザーID)のラベル
user_id_column = "hashvin"

# データの読み込み
normal_df = read_gz_files(normal_folder, user_id_column)
anormal_df = read_gz_files(anomaly_folder, user_id_column)

print(f"Number of normal users: {len(normal_df)}")
print(f"Number of anomaly users: {len(anormal_df)}")

# 故障df定義
failure_range_list = read_excel(excel_file_path)
anormal_df = extract_failure_data(anormal_df, failure_range_list)

# 抽出されたIGサイクルの出力
print_extracted_ig_cycles(anormal_df)

# 故障df,正常dfの各VINのtotal_ig_cycleの重複のないリスト取得
failure_unique_ig_cycles = {}
for vin, dfs in anormal_df.items():
    failure_unique_ig_cycles[vin] = pd.concat(dfs)['total_ig_cycle'].unique().tolist()

failure_total_unique_ig_cycles = sum(len(ig_cycles) for ig_cycles in failure_unique_ig_cycles.values())
print(f"Total unique IG cycles in failure data: {failure_total_unique_ig_cycles}")

normal_unique_ig_cycles = {}
for vin, dfs in normal_df.items():
    normal_unique_ig_cycles[vin] = pd.concat(dfs)['total_ig_cycle'].unique().tolist()

normal_total_unique_ig_cycles = sum(len(ig_cycles) for ig_cycles in normal_unique_ig_cycles.values())
print(f"Total unique IG cycles in normal data: {normal_total_unique_ig_cycles}")

Reading files from /root/user/j0114690/EGT/data_processed/normal: 100%|██████████| 959/959 [08:05<00:00,  1.98file/s]


Elapsed time for reading files from /root/user/j0114690/EGT/data_processed/normal: 485.22 seconds


Reading files from /root/user/j0114690/EGT/data_processed/anormal: 100%|██████████| 40/40 [00:25<00:00,  1.59file/s]


Elapsed time for reading files from /root/user/j0114690/EGT/data_processed/anormal: 25.14 seconds
Number of normal users: 959
Number of anomaly users: 40
VIN: A004, Extracted IG Cycles: [120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140]
VIN: A014, Extracted IG Cycles: [312, 313, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332]
VIN: A016, Extracted IG Cycles: [633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 651, 652, 653]
VIN: A021, Extracted IG Cycles: [1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303]
VIN: A057, Extracted IG Cycles: [314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334]
VIN: A066, Extracted IG Cycles: [579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598]
VIN: A075, Extrac

In [4]:
# 分析対象ラベル
feature_labels = [
    ('atmospheric_pressure', 'PA'),
    #('accelerationpedal_position', 'AP'),
    #('soc_real', 'SOC'),
    ('vehiclespeed_eat', 'Vp'),
    ('enginespeed', 'Ne'),
    ('eng_torque', 'TrqE'),
    ('eng_power', 'Peng'),
    ('engine_coolant_temperature', 'Tw'),
    #('driveshaft_torque', 'TrqD'),
    ('aircon_outsidetemperature_degc', 'TA'),
    ('intake_air_temperature', 'INTA'),
    #('fuel_ratio', 'AF'),
    #('temp_exhaust_gas', 'TCT'),
]

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
pd.options.mode.copy_on_write = True

def determine_bin_edges():
    global bin_edges

    normal_dfs = [df for dfs in normal_df.values() for df in dfs]
    anormal_dfs = [df for dfs in anormal_df.values() for df in dfs]
    all_dfs = normal_dfs + anormal_dfs

    bin_specs = []
    max_bins = 0

    for label, prefix in feature_labels:
        all_values = pd.concat([pd.to_numeric(df[label], errors='coerce') for df in all_dfs if label in df.columns])

        min_value = all_values.min()
        max_value = all_values.max()
        lower_percentile = all_values.quantile(lower_percentile_value)
        upper_percentile = all_values.quantile(upper_percentile_value)
        
        bin_width = (max_value - min_value) / n_bins
        
        bin_edges[label] = [min_value + i * bin_width for i in range(n_bins + 1)]
        
        bin_specs.append([
            prefix,
            lower_percentile,
            upper_percentile,
            min_value,
            max_value
        ] + bin_edges[label])

        max_bins = max(max_bins, len(bin_edges[label]))

    bin_specs_columns = [
        'variable',
        f'{lower_percentile_value*100:.2f}%_lower',
        f'{upper_percentile_value*100:.2f}%_upper',
        'min',
        'max'
    ] + [f'bin_edge_{i}' for i in range(max_bins)]

    bin_specs_df = pd.DataFrame(bin_specs, columns=bin_specs_columns)

    bin_specs_df.to_csv('bin_specs.csv', index=False)


def get_histogram_suffix(bin_start, bin_end):
    return f"{bin_start:.2f}-{bin_end:.2f}"


def extract_features(df, vin, dc, is_normal=False):
    # (1-0) DC全長の計算
    df['trip_len'] = (df['elapsed_time'].max() - df['elapsed_time'].min())+ 3
    
    # 駆動状態と停止状態の判定
    df['Vp_driving_state'] = np.where(df['vehiclespeed_eat'] >= 1, 1, 0)
    df['Vp_stop_state'] = 1 - df['Vp_driving_state']
    df['Ne_driving_state'] = np.where(df['enginespeed'] >= 800, 1, 0)
    df['Ne_stop_state'] = 1 - df['Ne_driving_state']
    
    # アイドル状態の判定
    df['idle_state'] = np.where((df['vehiclespeed_eat'] < 1) & (df['enginespeed'] >= 800), 1, 0)
    
    # (1-1) 車速とエンジン回転数に対する特徴量の計算
    Vp_features = calculate_features(df, 'Vp_driving_state', 'Vp_stop_state', 'Vp')
    Ne_features = calculate_features(df, 'Ne_driving_state', 'Ne_stop_state', 'Ne')
    
    # (1-2) アイドリング条件を満たす時間の計算
    idle_sum = (df[df['idle_state'] == 1]['time_diff'].sum()).clip(0)
    idle_max = (df[df['idle_state'] == 1]['time_diff'].groupby((df['idle_state'] != df['idle_state'].shift()).cumsum()).sum().max()).clip(0) if df['idle_state'].sum() > 0 else 0
    
    # 特徴量をデータフレームにまとめる
    features = pd.DataFrame({
        'hashvin': vin,
        'total_ig_cycle': dc,
        #'state': 'normal' if is_normal else 'anormal',
        'trip_len': df['trip_len'].iloc[0],
        **Vp_features,
        **Ne_features,
        'idle_sum': idle_sum,
        'idle_max': idle_max
    }, index=[0])
    
    return features

def calculate_features(df, driving_state_col, stop_state_col, prefix):
    driv_sum = (df[df[driving_state_col] == 1]['time_diff'].sum()).clip(0)
    driv_max = (df[df[driving_state_col] == 1]['time_diff'].groupby((df[driving_state_col] != df[driving_state_col].shift()).cumsum()).sum().max()).clip(0) if df[driving_state_col].sum() > 0 else 0
    p_driv_sum = driv_sum / df['trip_len'].iloc[0]
    p_driv_max = driv_max / df['trip_len'].iloc[0] if driv_max > 0 else 0
    stop_sum = df['trip_len'].iloc[0] - driv_sum
    stop_max = (df[df[stop_state_col] == 1]['time_diff'].groupby((df[stop_state_col] != df[stop_state_col].shift()).cumsum()).sum().max()).clip(0) if df[stop_state_col].sum() > 0 else 0
    p_stop_sum = stop_sum / df['trip_len'].iloc[0]
    p_stop_max = stop_max / df['trip_len'].iloc[0] if stop_max > 0 else 0
    stop_count = ((df[driving_state_col] != df[driving_state_col].shift()) & (df[driving_state_col] == 0)).sum() + (df[driving_state_col].iloc[0] == 0)
    
    return {
        f'{prefix}_driv_sum': driv_sum,
        f'{prefix}_driv_max': driv_max,
        f'{prefix}_p_driv_sum': p_driv_sum,
        f'{prefix}_p_driv_max': p_driv_max,
        f'{prefix}_stop_sum': stop_sum,
        f'{prefix}_stop_max': stop_max,
        f'{prefix}_p_stop_sum': p_stop_sum,
        f'{prefix}_p_stop_max': p_stop_max,
        f'{prefix}_stop_count': stop_count
    }


def extract_features_2(df, vin, dc, total_trip_len, dc_count, total_run_distance):
    # 走行距離(Vp×時間ベース)
    run_distance = (df['vehiclespeed_eat'] * df['time_diff']).sum() /(60*60)

    # DC平均走行距離
    ave_run_distance = total_run_distance / dc_count

    # 燃料消費量
    fuel_consumption = df['fuel_consumption_qty'].iloc[-1] - df['fuel_consumption_qty'].iloc[0]

    # 環境湿度(推定)
    TA_mean = df['aircon_outsidetemperature_degc'].mean()
    RH_ini = df['vehicleinterior_humidity'].iloc[0]
    RH_end = df['vehicleinterior_humidity'].iloc[-1]
    RH_ave = df['vehicleinterior_humidity'].mean()
    RH_max = df['vehicleinterior_humidity'].max()
    RH_min = df['vehicleinterior_humidity'].min()
    RH_TA_ini = RH_ini / TA_mean if TA_mean !=0 else np.nan
    RH_TA_end = RH_end / TA_mean if TA_mean !=0 else np.nan
    RH_TA_ave = RH_ave / TA_mean if TA_mean !=0 else np.nan
    RH_TA_max = RH_max / TA_mean if TA_mean !=0 else np.nan
    RH_TA_min = RH_min / TA_mean if TA_mean !=0 else np.nan

    # 積分エンジン出力
    ENG_energy = (df['eng_power'] * df['time_diff']).sum() / 3600  # [kW] to [kWh]

    # エンジン出力あたりの燃料消費量
    BSFC = fuel_consumption / ENG_energy if ENG_energy > 0 else 0

    # 時間[h]あたりの始動回数
    ig_freq_time = dc_count / total_trip_len * 3600 if total_trip_len != 0 else 0

    # 走行距離あたりの始動回数
    ig_freq_distance = dc_count / total_run_distance if total_run_distance != 0 else np.nan

    # 特徴量をデータフレームにまとめる
    features_2 = pd.DataFrame({
        'run_distance': run_distance,
        'ave_run_distance': ave_run_distance,
        'fuel_consumption': fuel_consumption,
        'RH_ini': RH_ini,
        'RH_end': RH_end,
        'RH_ave': RH_ave,
        'RH_max': RH_max,
        'RH_min': RH_min,
        'RH_TA_ini': RH_TA_ini,
        'RH_TA_end': RH_TA_end,
        'RH_TA_ave': RH_TA_ave,
        'RH_TA_max': RH_TA_max,
        'RH_TA_min': RH_TA_min,
        'ENG_energy': ENG_energy,
        'BSFC': BSFC,
        'ig_freq_time': ig_freq_time,
        'ig_freq_distance': ig_freq_distance
    }, index=[0])

    return features_2


def extract_features_3(dfs, vin, dc):
    df = pd.concat(dfs)
    
    # ソーク時間の計算
    df_dc = df[df['total_ig_cycle'] == dc]
    if not df_dc.empty:
        start_time = df_dc['tsu_current_time'].min()
        end_time = df_dc['tsu_current_time'].max()
        
        prev_dc = dc - 1
        next_dc = dc + 1
        
        prev_end_time = df[df['total_ig_cycle'] == prev_dc]['tsu_current_time'].max() if prev_dc in df['total_ig_cycle'].values else None
        next_start_time = df[df['total_ig_cycle'] == next_dc]['tsu_current_time'].min() if next_dc in df['total_ig_cycle'].values else None
        
        soaktime_bf = (start_time - prev_end_time).total_seconds() if prev_end_time is not None else np.nan
        soaktime_af = (next_start_time - end_time).total_seconds() if next_start_time is not None else np.nan
        
        # ソーク時間が0以下または3年以上の場合はNaNを設定
        soaktime_bf = soaktime_bf if 0 < soaktime_bf <= 94608000 else np.nan
        soaktime_af = soaktime_af if 0 < soaktime_af <= 94608000 else np.nan
    else:
        soaktime_bf = np.nan
        soaktime_af = np.nan
    
    # 特徴量をデータフレームにまとめる
    features_3 = pd.DataFrame({
        'soaktime_bf': soaktime_bf,
        'soaktime_af': soaktime_af
    }, index=[0])
    
    return features_3


def extract_features_4(df, vin, dc):    
    features_4 = {}
    for label, prefix in feature_labels:
        if label in df.columns:
            #df[label] = pd.to_numeric(df[label], errors='coerce')
            features_4[f'{prefix}_ini'] = df[label].iloc[0]
            #features_4[f'{prefix}_end'] = df[label].iloc[-1]
            features_4[f'{prefix}_d'] = (df[label].iloc[-1] - df[label].iloc[0])
            features_4[f'{prefix}_ave'] = df[label].mean()
            features_4[f'{prefix}_SD'] = df[label].std()
            features_4[f'{prefix}_max'] = df[label].max()
            features_4[f'{prefix}_min'] = df[label].min()
    
    features_4 = pd.DataFrame(features_4, index=[0])
    
    return features_4


def extract_features_5(df, vin, dc):
    # EV区間の特定
    df['ENG_driving_state'] = np.where(df['enginespeed'] >= 800, 1, 0)
    df['EV_state'] = 1 - df['ENG_driving_state']
    
    # EV開始時間とEV終了時間の特定
    df['EV_start'] = (df['EV_state'] != df['EV_state'].shift(1)) & (df['EV_state'] == 1)
    df['EV_end'] = (df['EV_state'] != df['EV_state'].shift(-1)) & (df['EV_state'] == 1)
    
    # 最長EV区間の特定
    if df['EV_state'].sum() > 0:
        ev_start_times = df[df['EV_start']]['elapsed_time'].values
        ev_end_times = df[df['EV_end']]['elapsed_time'].values
        
        # EV開始時間とEV終了時間の数を揃える
        min_length = min(len(ev_start_times), len(ev_end_times))
        ev_start_times = ev_start_times[:min_length]
        ev_end_times = ev_end_times[:min_length]
        
        ev_intervals = pd.DataFrame({
            'start': ev_start_times,
            'end': ev_end_times
        })
        ev_intervals['duration'] = ev_intervals['end'] - ev_intervals['start']
        
        if not ev_intervals.empty:
            max_ev_interval = ev_intervals.loc[ev_intervals['duration'].idxmax()]
            max_ev_start = max_ev_interval['start']
            max_ev_end = max_ev_interval['end']
        else:
            max_ev_start = np.nan
            max_ev_end = np.nan
    else:
        max_ev_start = np.nan
        max_ev_end = np.nan
    
    # EV終了行(時間)の特定
    if not np.isnan(max_ev_end):
        max_ev_end_index = df[df['elapsed_time'] == max_ev_end].index[0]
    else:
        max_ev_end_index = df.index[-1] if df['EV_state'].iloc[-1] == 1 else np.nan
    
    # (5-1) 最長EV区間前後の変化差分  
    features_5_1 = {}
    for label, prefix in feature_labels:
        if label in df.columns and not np.isnan(max_ev_end_index):
            max_ev_end_value = df.loc[max_ev_end_index, label]
            
            # 最長EV終了後の変化差分
            after_ev_mask = (df.index > max_ev_end_index) & (df.index <= max_ev_end_index + 15)
            if after_ev_mask.sum() > 0:
                after_ev_diffs = df.loc[after_ev_mask, label] - max_ev_end_value
                max_diff_index = after_ev_diffs.abs().idxmax()
                features_5_1[f'{prefix}_aEV_5tsD'] = after_ev_diffs.loc[max_diff_index]
            else:
                features_5_1[f'{prefix}_aEV_5tsD'] = 0
            
            # 最長EV終了前の変化差分
            before_ev_mask = (df.index >= max_ev_end_index - 15) & (df.index < max_ev_end_index)
            if before_ev_mask.sum() > 0:
                before_ev_diffs = df.loc[before_ev_mask, label] - max_ev_end_value
                max_diff_index = before_ev_diffs.abs().idxmax()
                features_5_1[f'{prefix}_eEV_5tsD'] = before_ev_diffs.loc[max_diff_index]
            else:
                features_5_1[f'{prefix}_eEV_5tsD'] = 0
        else:
            features_5_1[f'{prefix}_aEV_5tsD'] = 0
            features_5_1[f'{prefix}_eEV_5tsD'] = 0
    
    # (5-2) 最長EV区間中の基本統計量
    features_5_2 = {}
    for label, prefix in feature_labels:
        if label in df.columns and not np.isnan(max_ev_start) and not np.isnan(max_ev_end):
            ev_mask = (df['elapsed_time'] >= max_ev_start) & (df['elapsed_time'] <= max_ev_end)
            ev_data = df.loc[ev_mask, label]
            features_5_2[f'{prefix}_EV_ini'] = ev_data.iloc[0] if len(ev_data) > 0 else np.nan
            features_5_2[f'{prefix}_EV_end'] = ev_data.iloc[-1] if len(ev_data) > 0 else np.nan
            features_5_2[f'{prefix}_EV_d'] = ev_data.iloc[-1] - ev_data.iloc[0] if len(ev_data) > 0 else np.nan
            features_5_2[f'{prefix}_EV_ave'] = ev_data.mean() if len(ev_data) > 0 else np.nan
            features_5_2[f'{prefix}_EV_SD'] = ev_data.std() if len(ev_data) > 0 else np.nan
            features_5_2[f'{prefix}_EV_max'] = ev_data.max() if len(ev_data) > 0 else np.nan
            features_5_2[f'{prefix}_EV_min'] = ev_data.min() if len(ev_data) > 0 else np.nan
        else:
            features_5_2[f'{prefix}_EV_ini'] = np.nan
            features_5_2[f'{prefix}_EV_end'] = np.nan
            features_5_2[f'{prefix}_EV_d'] = np.nan
            features_5_2[f'{prefix}_EV_ave'] = np.nan
            features_5_2[f'{prefix}_EV_SD'] = np.nan
            features_5_2[f'{prefix}_EV_max'] = np.nan
            features_5_2[f'{prefix}_EV_min'] = np.nan
    
    # (5-3) 最長EV区間開始前のENG駆動期間中の積分エンジン出力の計算
    if not np.isnan(max_ev_start):
        before_ev_mask = (df['elapsed_time'] < max_ev_start) & (df['ENG_driving_state'] == 1)
        if before_ev_mask.sum() > 0:
            before_ev_data = df.loc[before_ev_mask, ['eng_power', 'time_diff']]
            features_5_2['ENG_energy_bEV'] = (before_ev_data['eng_power'] * before_ev_data['time_diff']).sum() / 3600
        else:
            features_5_2['ENG_energy_bEV'] = 0
    else:
        features_5_2['ENG_energy_bEV'] = 0
    
    features_5 = pd.concat([pd.DataFrame(features_5_1, index=[0]), pd.DataFrame(features_5_2, index=[0])], axis=1)
    
    return features_5


def extract_features_6(df, vin, dc, total_trip_len, hist_acc_dict, dc_count):
    features_6 = {}

    for label, prefix in feature_labels:
        if label in df.columns and label in bin_edges:
            # numpy.histogramを使用してヒストグラムを計算
            hist, edges = np.histogram(df[label], bins=bin_edges[label])
            
            # ヒストグラムのbinラベルを生成 (最小と最大のエッジを含む)
            bin_suffixes = [get_histogram_suffix(edges[i], edges[i+1]) for i in range(len(edges)-1)]

            # 各binの最大連続使用時間(LSPV)を計算
            bin_indices = pd.cut(df[label], bins=edges, labels=False, include_lowest=True, right=False)
            bin_counts = bin_indices.groupby((bin_indices != bin_indices.shift()).cumsum()).transform('size')
            max_bin_counts = bin_counts.groupby(bin_indices).max()

            for i, (freq, suffix) in enumerate(zip(hist, bin_suffixes)):
                # hist使用時間
                #features_6[f"{prefix}_Time:{suffix}"] = freq * 3
                features_6[f"{prefix}_pTime:{suffix}"] = freq * 3 / df['trip_len'].iloc[0]

                # hist累積使用時間
                if f"{prefix}_acc:{suffix}" not in hist_acc_dict:
                    hist_acc_dict[f"{prefix}_acc:{suffix}"] = freq * 3
                else:
                    hist_acc_dict[f"{prefix}_acc:{suffix}"] += freq * 3
                
                # hist累積使用時間割合
                #features_6[f"{prefix}_Acc:{suffix}"] = hist_acc_dict[f"{prefix}_acc:{suffix}"]
                features_6[f"{prefix}_pAcc:{suffix}"] = hist_acc_dict[f"{prefix}_acc:{suffix}"] / total_trip_len
                features_6[f"{prefix}_dcAve:{suffix}"] = hist_acc_dict[f"{prefix}_acc:{suffix}"] / dc_count

                # MaxTime(=LSPV)
                # if i in max_bin_counts.index:
                #     features_6[f"{prefix}_MaxTime:{suffix}"] = max_bin_counts[i] * 3
                # else:
                #     features_6[f"{prefix}_MaxTime:{suffix}"] = 0

                #features_6[f"{prefix}_pMaxTime:{suffix}"] = features_6[f"{prefix}_MaxTime:{suffix}"] / df['trip_len'].iloc[0]

    features_6 = pd.DataFrame(features_6, index=[0])

    return features_6, hist_acc_dict



In [6]:
#bin数,分位点確認
n_bins = 20
lower_percentile_value = 0.01 #0.0015
upper_percentile_value = 0.99 #0.9985

#bin範囲計算
bin_edges = {}
determine_bin_edges()

In [ ]:
from tqdm import tqdm

# 正常,異常 特徴計算
normal_features = []
anormal_features = []

for vin, dfs in tqdm(normal_df.items(), desc="Normal features", unit="VIN"):
    df_concat = pd.concat(dfs)
    total_trip_len = 0
    dc_count = 0
    total_run_distance = 0
    hist_acc_dict = {}
    for dc in normal_unique_ig_cycles[vin]:
        df = df_concat[df_concat['total_ig_cycle'] == dc]
        df['aircon_outsidetemperature_degc'] = pd.to_numeric(df['aircon_outsidetemperature_degc'], errors='coerce')
        df['vehicleinterior_humidity'] = pd.to_numeric(df['vehicleinterior_humidity'], errors='coerce')
        features = extract_features(df, vin, dc, is_normal=True)
        total_trip_len += df['trip_len'].iloc[0]
        dc_count += 1
        ave_trip_len = total_trip_len/dc_count
        total_run_distance += (df['vehiclespeed_eat'] * df['time_diff']).sum() /(60*60)
        features_2 = extract_features_2(df, vin, dc, total_trip_len, dc_count, total_run_distance)
        features_3 = extract_features_3(dfs, vin, dc)
        features_4 = extract_features_4(df, vin, dc)
        features_5 = extract_features_5(df, vin, dc)
        features_6, hist_acc_dict = extract_features_6(df, vin, dc, total_trip_len, hist_acc_dict, dc_count)
        features = pd.concat([features, features_2, features_3, features_4, features_5, features_6], axis=1)
        #features.insert(features.columns.get_loc('trip_len') + 1, 'total_trip_len', total_trip_len)
        features.insert(features.columns.get_loc('trip_len') + 1, 'ave_trip_len', ave_trip_len)
        normal_features.append(features)

for vin, dfs in tqdm(anormal_df.items(), desc="Anomal features", unit="VIN"):
    df_concat = pd.concat(dfs)
    total_trip_len = 0
    dc_count = 0
    total_run_distance = 0
    hist_acc_dict = {}
    for dc in failure_unique_ig_cycles[vin]:
        df = df_concat[df_concat['total_ig_cycle'] == dc]
        df['aircon_outsidetemperature_degc'] = pd.to_numeric(df['aircon_outsidetemperature_degc'], errors='coerce')
        df['vehicleinterior_humidity'] = pd.to_numeric(df['vehicleinterior_humidity'], errors='coerce')
        features = extract_features(df, vin, dc, is_normal=False)
        total_trip_len += df['trip_len'].iloc[0]
        dc_count += 1
        ave_trip_len = total_trip_len/dc_count
        total_run_distance += (df['vehiclespeed_eat'] * df['time_diff']).sum() /(60*60)
        features_2 = extract_features_2(df, vin, dc, total_trip_len, dc_count, total_run_distance)
        features_3 = extract_features_3(dfs, vin, dc)
        features_4 = extract_features_4(df, vin, dc)
        features_5 = extract_features_5(df, vin, dc) 
        features_6, hist_acc_dict = extract_features_6(df, vin, dc, total_trip_len, hist_acc_dict, dc_count)
        features = pd.concat([features, features_2, features_3, features_4, features_5, features_6], axis=1)
        #features.insert(features.columns.get_loc('trip_len') + 1, 'total_trip_len', total_trip_len)
        features.insert(features.columns.get_loc('trip_len') + 1, 'ave_trip_len', ave_trip_len)
        anormal_features.append(features)

# 特徴量データフレームの結合
normal_features_df = pd.concat(normal_features, ignore_index=True)
anormal_features_df = pd.concat(anormal_features, ignore_index=True)

# 特徴量数、サンプルサイズ
num_features = normal_features_df.shape[1] - 2
total_samples = normal_features_df.shape[0] + anormal_features_df.shape[0]
feature_sample_ratio = num_features / total_samples * 100

print(f"Number of features: {num_features}")
print(f"Total sample size: {total_samples}")
print(f"Feature/sample: {feature_sample_ratio:.2f}% (target:5%)")

# NaNを各列の平均値で穴埋め
for df in [normal_features_df, anormal_features_df]:
    for column in df.columns:
        if df[column].isnull().any():
            mean_value = df[column].mean(skipna=True)
            df[column] = df[column].apply(lambda x: mean_value if pd.isna(x) else x)


Normal features:  18%|█▊        | 170/959 [50:24<2:14:14, 10.21s/VIN]

In [ ]:
# 特徴量の順番を入れ替え
def reorder_features(df):
    reordered_features = []
    
    for feature_type in ['Time', 'pTime', 'Acc', 'pAcc', 'dcAve', 'MaxTime', 'pMaxTime']:
        for label, prefix in feature_labels:
            for col in df.columns:
                if col.startswith(f"{prefix}_{feature_type}"):
                    reordered_features.append(col)
    
    existing_columns = [col for col in df.columns if col not in reordered_features]
    reordered_df = df[existing_columns + reordered_features]
    
    return reordered_df

normal_features_df = reorder_features(normal_features_df)
anormal_features_df = reorder_features(anormal_features_df)

In [ ]:
# CSVファイルとして出力
normal_features_df.to_csv('normal_features.csv', index=False)
anormal_features_df.to_csv('anormal_features.csv', index=False)